### Script to download Southern Neveda Restaurant dataset on health violations

In [13]:
import os
import sys
import urllib
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib
import psycopg2
from sqlalchemy import create_engine

sys.path.append('../')
from envir import config
%matplotlib inline

In [14]:
engine = create_engine('postgresql+psycopg2:///yelp_abcs')
con = engine.connect()

In [3]:
! curl https://www.southernnevadahealthdistrict.org/restaurants/download/restaurants.zip > nevadaRestaurants.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.2M  100 11.2M    0     0  11.2M      0  0:00:01  0:00:01 --:--:-- 7268k


In [4]:
# move into shared data folder
os.system('mv nevadaRestaurants.zip '+ config.shared)
os.system('mkdir nevadaRestaurants')

256

In [5]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile(config.shared+'/nevadaRestaurants.zip', 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall(config.shared+'/nevadaRestaurants')

In [3]:
# get every file in the unzip folder
f = []
for root, dirs, files in os.walk(config.shared+'/nevadaRestaurants'):
    for filename in files:
        print(filename)
        f.append(filename)

restaurant_categories.csv
restaurant_cities.csv
restaurant_establishments.csv
restaurant_inspection_types.csv
restaurant_inspections.csv
restaurant_inspections_stanford.csv
restaurant_pe_category_xref.csv
restaurant_serials.csv
restaurant_violations.csv
restaurants_serials.csv


In [7]:
# download a newer version of restaurants violations from open data
# https://opendataportal-lasvegas.opendata.arcgis.com/datasets/restaurant-inspection-violation-codes/data?page=9
!curl https://opendata.arcgis.com/datasets/9abf2b74783e49f4949afc06839860a7_0.csv > restaurant_violations.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6903    0  6903    0     0   6903      0 --:--:-- --:--:-- --:--:-- 27722


In [8]:
# move into shared data folder
os.system('mv restaurant_violations.csv '+ config.shared+'/nevadaRestaurants/')

0

In [4]:
list_of_dfs = {}
df_names = []

for filename in f:
    file = filename[:-4]
    df_names.append(file)
    print(file)
    list_of_dfs[file] = pd.read_csv(config.shared+'nevadaRestaurants/'+filename, sep=";")
    
list_of_dfs['restaurant_violations'] = pd.read_csv(config.shared+'nevadaRestaurants/'+'restaurant_violations.csv',
                                                   sep=",")


restaurant_categories
restaurant_cities
restaurant_establishments
restaurant_inspection_types
restaurant_inspections
restaurant_inspections_stanford
restaurant_pe_category_xref
restaurant_serials
restaurant_violations
restaurants_serials


In [21]:
list_of_dfs['restaurant_violations'].columns = map(str.lower, list_of_dfs['restaurant_violations'])
list_of_dfs['restaurant_violations']

,violation_id,violation_code,violation_demerits,violation_description,objectid
0,1,1,6,Food not obtained from approved sources and/or...,1
1,15,15,6,Sewage not disposed into public sewer or appro...,2
2,16,16,6,No hot and cold running water as required and/...,3
3,17,17,2,Fruits and vegetables improperly washed prior ...,4
4,18,18,1,Foods not stored off the floor.,5
...,...,...,...,...,...
85,38,38,1,Living quarters not completely separated from ...,86
86,61,6-1,6,Poultry,87
87,62,6-2,6,Ground,88
88,63,6-3,6,Pork and/or any food containing pork,89


In [19]:
list_of_dfs.keys()

dict_keys(['restaurant_categories', 'restaurant_cities', 'restaurant_establishments', 'restaurant_inspection_types', 'restaurant_inspections', 'restaurant_inspections_stanford', 'restaurant_pe_category_xref', 'restaurant_serials', 'restaurant_violations', 'restaurants_serials'])

In [26]:
for i in range(len(df_names)):
    print(df_names[i])
    try:
        list_of_dfs[df_names[i]].iloc[:, :-1].to_sql('nevada_'+df_names[i], engine, schema='stag',
                                                     index=False)
    except:
        print("table already exists")

restaurant_categories
table already exists
restaurant_cities
table already exists
restaurant_establishments
table already exists
restaurant_inspection_types
table already exists
restaurant_inspections
table already exists
restaurant_inspections_stanford
table already exists
restaurant_pe_category_xref
table already exists
restaurant_serials
table already exists
restaurant_violations
table already exists
restaurants_serials
table already exists
